<a href="https://colab.research.google.com/github/PogadaevEA/cian/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22CIAN_Cleaner_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#0. Install libraries
!pip install -U git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-34tjj1jp
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-34tjj1jp
  Created wheel for efficientnet: filename=efficientnet-1.1.1-cp36-none-any.whl size=18421 sha256=65ac45ed23d964fd3732d20c0269bf9d65c701f15aa0d93dc74af23da8bf638b
  Stored in directory: /tmp/pip-ephem-wheel-cache-r3oa1hkt/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet
  Found existing installation: efficientnet 1.1.1
    Uninstalling efficientnet-1.1.1:
      Successfully uninstalled efficientnet-1.1.1


In [ ]:
#1. Loading BeautifulSoup and test request
from bs4 import BeautifulSoup
import requests


def load_flat_dicts(url):
    cian_html = requests.get(url).text
    soup = BeautifulSoup(cian_html)


    flats_dict = {}
    flats = soup.find_all('div', {'data-name': 'TopOfferCard'})
    flats += soup.find_all('div', {'data-name': 'OfferCard'})
    for f in flats:
        flat_imgs = []

        additional_imgs = f.find_all('img', {'data-name': 'GalleryImage'})
        flat_imgs.append(f.find('img')['src'])

        for fa in additional_imgs:
            flat_imgs.append(fa['src'])

        links = f.find_all('a', {'target': '_blank'})
        for a in links:
            if 'https://www.cian.ru/rent/flat/' or 'https://www.cian.ru/sale/flat/' in a['href']:
                if '/cat.php?' not in a['href']:
                  flats_dict[a['href']] = flat_imgs
                  break



    return flats_dict
flats_dict = load_flat_dicts('https://www.cian.ru/cat.php?currency=2&deal_type=rent&engine_version=2&maxprice=25000&minprice=15000&offer_type=flat&region=1&room1=1&room2=1&type=4')
print('Testing done!')

Testing done!


In [ ]:
%%time
#2. Building model and predict one image

from tensorflow.keras.models import load_model, Model
from tensorflow.keras.utils import get_file
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
import efficientnet.tfkeras as efn
import io
from urllib.request import urlopen

#0.96
def build_model_efficientnet():
    pretrained_model = efn.EfficientNetB0(weights='imagenet', include_top=False)
    pretrained_model.trainable = False
    x = pretrained_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=pretrained_model.input, outputs=predictions)
    
    # lr=1e-4
    model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])
    model.summary()
    return model


### LOADING MODEL
model = build_model_efficientnet()
# model = load_model('./cian_model_01_B0.h5')
weights_path = get_file(fname='cian_model_01_B0.h5', origin='https://fancyshot.com/wp-content/uploads/model/cian_model_01_B0.h5')
model.load_weights(weights_path)
print('Model Loaded!')


def _fast_expand(img):
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img


def pseudo_download_image(url):
#     print(f'[INFO] Downloading {url}')
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    return image



def predict_image(url):
    img_size = 320
#     open_cv_image = cv2.imread(img_path)
    open_cv_image = pseudo_download_image(url)
    open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB)
    test_image = cv2.resize(open_cv_image, (img_size, img_size))
    orig_image = _fast_expand(test_image)
    result_orig = model.predict(orig_image, batch_size=1)
    
#     classes = ['bad', 'good']
    # result_idx = np.argmax(result_orig)
    result_val = list(result_orig[0])
    
    return result_val

#Testing prediction
testing_res = predict_image('https://cdn-p.cian.site/images/8/766/147/kvartira-moskva-lomonosovskiy-prospekt-741667887-4.jpg')
print(f'Tested value: {testing_res}')

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
_______________________________________________________________________________________

HTTPError: ignored

In [ ]:
#3. Predict for all flats from result page
from IPython.core.display import display, HTML

def show_results(scores, threshold=0.5):
    out_html = ['''<table border="1" cellpadding="5" width="100%" class="table table-striped">
                  <tr>
                  <th>Фотография</th>
                  <th>Ссылка</th>
                  <th>Оценка</th>
                  </tr>
                  <tbody>
                ''']


    for f, scr in scores.items():
        img_thumb = scr['thumb']
        f_score = scr['score']
        class_idx = f_score.index(max(f_score))
        if class_idx == 1 and max(f_score) >= threshold:
            out_html.append(f'''<tr>
            <td><a href="{f}" target="_blank">
            <img src="{img_thumb}" width=250 /></a>
            </td>
            <td><a href="{f}" target="_blank">{f}</a></td>
            <td>{f_score}</td></tr>''')

    out_html.append('</tbody></table>')
    print('->>> RESULT STARTS HERE <<<-')
    display(HTML(' '.join(out_html)))

    
### MAIN PART

def FilterAllShitPlease(cian_url, threshold=0.5):
  CIAN_URL = cian_url    
  result_scores = {}
  flats_dict = load_flat_dicts(cian_url)

  for f, urls in flats_dict.items():
      data = {'score': [0, 0], 'thumb': urls[0]}
      total_score = [0, 0]
      for url in urls:
          data['score'] = predict_image(url)
          total_score = [a + b for a, b in zip(total_score, data['score'])]

      final_score = [x/len(urls) for x in total_score]
      data['score'] = final_score
      result_scores[f] = data
      classes = ['bad', 'good']
      print(f'Score for {f} -> {final_score}: This is a {classes[final_score.index(max(final_score))]} flat.')
  
  #ПОКАЗАТЬ ТОЛЬКО ОТСЕЯННЫЕ КВАРТИРЫ. threshold – порог качество от 0 до 1
  show_results(result_scores, threshold=threshold)
    

### MAGIC HAPPENS HERE

In [ ]:
### MAGIC HAPPENS HERE
### Здесь просто нужно заменить ссылку на вашу выдачу. Пройдите по ней, чтобы было понятно, что это выдача от cian.ru

CIAN_URL = 'https://www.cian.ru/cat.php?currency=2&deal_type=rent&engine_version=2&maxprice=55000&minprice=40000&offer_type=flat&region=1&room1=1&room2=1&room3=1&type=4'
THRESHOLD = 0.66


FilterAllShitPlease(CIAN_URL, threshold=THRESHOLD)

NameError: ignored